<a href="https://colab.research.google.com/github/mikebmanzon/birdcall/blob/main/Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks such as summarization is hard enoough.

In [499]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import lil_matrix
from scipy.sparse import hstack
import numpy as np

In [500]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [513]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  #Your code here

  vectorizer = CountVectorizer(token_pattern=r'\b\S*[a-z]+\S*\b')
  texts = vectorizer.fit_transform([doc['text'] for doc in corpus])
  vocab = vectorizer.get_feature_names()
  genres = [doc['genre'] for doc in corpus]

  return texts,genres,vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createInterestingFeatures(corpus):
  #Your code here
  """
  reg_pattern = r'\b\S*[a-z]\S*\b'
  # tried using prefix and suffix patterns as well, but realized that the
  # shakespeare corpus doesn't include any hyphens
  prefix_pattern = r'\b\S*[a-z]\S*-?\b'
  suffix_pattern = r'\b-?\S*[a-z]\S*?\b'

  # tfidf_vectorizer had MUCH worse accuracy (I'm talking in the 50% range)
  tfidf_vectorizer = TfidfVectorizer(token_pattern=reg_pattern)

  # tried considering unigrams and bigrams as features, but accuracy suffered
  # in L1 and L2 and nothing notable was revealed.
  bigram_vectorizer = CountVectorizer(ngram_range=(1,2), token_pattern=reg_pattern, min_df=3, max_df=.7)
  """
  # the most success came from using only unigram token counts as features, 
  # and setting minimum and maximum document frequencies. This helped
  # avoid having common words and play-specific names/locations take too much
  # importance. I also tried binary counts for each feature. Names no longer 
  # appeared as most indicative, which was nice, but archaic words that should
  # be indicative of tragedy (like 'murther') did not show up
  # when using binary counts. I did find it interesting that 'tragedy' was only 
  # a highly informative word when using binary counts, but many of the same words
  # appeared as highly indicative of both genres in L1.
  #
  # The clearest dichotomy to me was tradegies featuring more 'mature' ideas
  # and occupations (nations like 'rome', powerful people like
  # 'general' and 'senators', lofty idealized roles like 'poet'), 
  # and more gruesome topics ('slain', 'murther', 'ghost', 'witch'),
  # while comedies focused on more lay occupations ('shepherd', 
  # 'clown','tailor') and fantastical elements ('jest', 'fairy'). Even their
  # royalty was more lighthearted ('princess' and 'duke'). Some words were still
  # considered highly informative for both genres, but there were much fewer of
  # them, and they were only at the tail end of the informative lists in L1.
  #
  # Note: min_df=3 and max_df=.6 gave me the highest accuracy, but names
  # were given too much weight for my taste. min_df=0.3 cut out some names,
  # and included more militaristic terms for tragedy, while including 
  # prioritizing words like 'dance' and 'humour' for comedy. However, there
  # are only 26 documents, and I felt that requiring a term to appear in at 
  # least 8 of them was a high bar. min_df=6 gave what I though to be the 
  # most interesting mix of indicators for tragedy and comedy.
  reg_pattern = r'\b\S*[a-z]\S*\b'

  # used stop words to get rid of words I felt were rated to highly, so I 
  # could find more interesting ones (mainly names)
  STOP_WORDS = ['antonio', 'john', 'diana', 'corn', 'jew', 'fish', 'ship', 'bottom']
  vectorizer = CountVectorizer(token_pattern=reg_pattern, min_df=6, max_df=.7, stop_words=STOP_WORDS)
  texts = vectorizer.fit_transform([doc['text'] for doc in corpus])
  vocab = vectorizer.get_feature_names()
  genres = [doc['genre'] for doc in corpus]

  return texts,genres,vocab

In [502]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  

In [503]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [504]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.653846
The most informative terms for pos are: ['you', 'helena', 'duke', 'prospero', 'sir', 'i', 'leontes', 'a', 'private', 'preserving', 'preservers', 'preserver', 'preserved', 'preserve', 'preservative', 'president', 'preservation', 'presents', 'presentment', 'presently']
The most informative terms for neg are: ['him', 's', 'iago', 'imogen', 'o', 'brutus', 'lear', 'ham', 'and', 'what', 'rom', 'pretending', 'preserving', 'preservers', 'preserver', 'preserved', 'pretext', 'preserve', 'preservative', 'preservation']
----------L2 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['i', 'you', 'duke', 'prospero', 'a', 'helena', 'your', 'antonio', 'sir', 'leontes', 'hermia', 'for', 'lysander', 'ariel', 'sebastian', 'demetrius', 'camillo', 'stephano', 'me', 'parolles']
The most informative terms for neg are: ['iago', 'othello', 's', 'him', 'imogen', 'what', 'lear', 'brutus', 'his', '

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [514]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.846154
The most informative terms for pos are: ['duke', 'signior', 'widow', 'fairy', 'shepherd', 'princess', 'cell', 'page', 'clown', 'preserve', 'presume', 'prevail', 'presents', 'prevent', 'presented', 'press', 'abandon', 'price', 'prey', 'prick']
The most informative terms for neg are: ['general', 'castle', 'daughters', 'roman', 'nurse', 'poet', 'painter', 'rome', 'caesar', 'prerogative', 'prepared', 'prepare', 'presents', 'preserve', 'press', 'presented', 'abandon', 'prevail', 'preparation', 'prevent']
----------L2 Norm-----------
The model's average accuracy is 0.846154
The most informative terms for pos are: ['duke', 'signior', 'widow', 'shepherd', 'clown', 'count', 'fairy', 'chain', 'princess', 'ducats', 'bond', 'merchant', 'wood', 'jest', 'wall', 'page', 'bottle', 'burden', 'dance', 'cell']
The most informative terms for neg are: ['general', 'murther', 'nurse', 'castle', 'tonight', 'ghost', 'rome', 'roman', 'painter